In [ ]:
import os
import time
import json
import datetime
import numpy as np
import pandas as pd
import pickle
from datetime import datetime

In [ ]:
import seaborn as sns
from plotly import express as px
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [ ]:
%%time
submit = json.load(open('./data/sample_submission/sample_submission.json', 'r', encoding='utf8')) 
df_dic = pickle.load(open('./mydata/df_dic.plk','rb'))
plc_lst = pickle.load(open('./mydata/plc_lst.plk','rb'))
fct_lst = ['pH', 'COD', 'SS', 'N', 'P', 'T']

In [ ]:
%%time
''' fillna : 단순 missing value 채움'''
for plc in plc_lst:
    df = df_dic[plc]
    df = df.fillna(method='ffill')
    df = df.fillna(method='bfill')    
    df_dic[plc] = df.dropna()
    
    if df_dic[plc].isna().any().any():
        print(plc)

In [ ]:
%%time
for i in tqdm(plc_lst):
    plc_df= df_dic[i]
    for f in fct_lst:
        df= plc_df[f]
        fig, ax = plt.subplots(1, 1, figsize = (12,6))


        g1 = sns.lineplot(x= range(len(df)), y= df.values ,ax = ax)
        g1.set(xticklabels=[])
        g1.set(title=f'{i}__{f}')
        g1.set(xlabel=None)
   
        ff = g1.get_figure()
        ff.savefig(f'./plot_data/{f.lower()}/{i}_{f}.png') 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (12,6))


g1 = sns.lineplot(x= range(len(df)), y= df.values ,ax = ax)
g1.set(xticklabels=[])
g1.set(title=f'{i}__{f}')
g1.set(xlabel=None)

In [ ]:
''' 전체 통채로 만들기 '''
plc_sorted_lst = sorted(plc_lst)

dfs = []
for lo in plc_sorted_lst:
    df_temp = df_dic[lo]
    df_temp['loc'] = lo
    dfs.append(df_temp)
dfall = pd.concat(dfs)

In [ ]:
''' 기록이 시작되는 공통 날짜 구하기 '''
dates = []
for lo in plc_sorted_lst:
    df_temp = df_dic[lo]
    dates.append(df_temp.ds[0])
mx_date= max(dates)
print(mx_date)  

In [ ]:
''' 최소 날짜가 포함되어 있는지 확인. -> 모두 포함되어 있다. '''
for lo in plc_sorted_lst:
    df_temp = df_dic[lo]
    if ~(df_temp.ds == mx_date).any():
        print(lo)

In [ ]:
''' datetime  열 만들기 '''
dt_list =  [ datetime.strptime(x, "%Y%m%d")for x in  list(dfall['ds'])]
dfall['dst'] = dt_list

''' 공통 날짜로 맞추기 '''
dfall_comm = dfall.loc[dfall.dst >= datetime(2015,4,23)]
dfall_comm = dfall_comm.reset_index(drop=True)

In [ ]:
''' 모든 업장의 데이터 길이가 같은지 확인 -> 1015 개로 똑같다. '''
res = dfall_comm.groupby(by = 'loc').size()

In [ ]:
for col in ['pH', 'COD', 'SS', 'N', 'P', 'T']:
    dfall_comm[col] = pd.to_numeric(dfall_comm[col])

In [ ]:
pickle.dump(dfall_comm , open('./mydata/dfall_comm.plk','wb'))

---
## 6개의 피쳐끼리 corr 보기 

In [ ]:
dfall_comm.columns

In [ ]:
df_corr = dfall_comm[['pH', 'COD', 'SS', 'N', 'P', 'T']]

In [ ]:
df_corr.corr('pearson')

---
## 시각화로 확인

In [ ]:
%%time
''' fillna : 단순 missing value 채움'''
for plc in plc_lst:
    df = df_dic[plc]
    df = df.fillna(method='ffill')
    df = df.fillna(method='bfill')    
    df_dic[plc] = df.dropna()
    
    if df_dic[plc].isna().any().any():
        print(plc)

In [ ]:
dfs = []
for lo in plc_sorted_lst:
    df_temp = df_dic[lo]
    df_temp['loc'] = lo
    dfs.append(df_temp)

In [ ]:
dfall = pd.concat(dfs)

In [ ]:

fig = px.line(dfall, x="ds", y="pH", color='loc')
fig.show()